# Import Libraries & Images

In [3]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import geopandas as gpd
import rasterio
import rasterio.plot

In [8]:
# Each JPG file showcases a circular snapshot of the land use raster centered at the centroids of the following HUCs (centroid_long, centroid_lat, NAME):

# NOTE: Each raster circle has a roughly 1 mile radius and the HUCs are arranged in descending order of centroid_long value
# -75.963985 39.362469 Lower Sassafras River
# -76.151131 39.173516 Langford Creek
# -76.479104 38.991919 Severn River-Frontal Chesapeake Bay
# -76.498649 38.505046 Parkers Creek-Frontal Chesapeake Bay
# -76.498173 38.411304 Saint Leonard Creek-Patuxent River
# -76.130023 38.274951 Fox Creek-Honga River
# -76.371830 38.187095 Saint Jerome Creek-Frontal Chesapeake Bay
# -76.262377 37.845446 Cockrell Creek-Frontal Chesapeake Bay

img_list = ["C:\\Users\\limzh\\OneDrive\\Pictures\\Lower Sassafras River HUC.JPG", 
            "C:\\Users\\limzh\\OneDrive\\Pictures\\Langford Creek HUC.JPG", 
            "C:\\Users\\limzh\\OneDrive\\Pictures\\Severn River-Frontal Chesapeake Bay HUC.JPG", 
            "C:\\Users\\limzh\\OneDrive\\Pictures\\Parkers Creek-Frontal Chesapeake Bay HUC.JPG", 
            "C:\\Users\\limzh\\OneDrive\\Pictures\\Saint Leonard Creek-Patuxent River HUC.JPG", 
            "C:\\Users\\limzh\\OneDrive\\Pictures\\Fox Creek-Honga River HUC.JPG", 
            "C:\\Users\\limzh\\OneDrive\\Pictures\\Saint Jerome Creek-Frontal Chesapeake Bay HUC.JPG", 
            "C:\\Users\\limzh\\OneDrive\\Pictures\\Cockrell Creek-Frontal Chesapeake Bay HUC.JPG"]

# Storing Color Data & Setting Up Percentage Function

In [76]:
# COLOR CHART
black = ['Impervious Road', 0, 0, 0]
maroon = ['Impervious Non-road', 115, 0 ,0]
pink = ['Fractional Turf', 255, 190, 232]
orange = ['Cropland/Pasture', 230, 152, 0]
purple = ['Fractional Impervious', 197, 0, 255]
mint = ['Tidal Wetland', 0,255,197]
dark_mint = ['Floodplain Wetland',0,230,168]
green = ['Forest', 38, 115, 0]
brown = ['Mixed Open', 168, 112, 1]
light_green = ['Tree Cover Over Turf', 170, 255, 1]
neon_green = ['Tree Cover Over Impervious', 85, 255, 0]
blue = ['Water', 0, 113, 254]
color_array = [black, maroon, pink, orange, purple, mint, dark_mint, green, brown, light_green, neon_green, blue]

diff = 15 # account for color grading differentials in color map

In [85]:
data_list = []

# Removes surrounding margin of raster JPG
def remove_margin(img):
    white_boundary = [([245, 245, 245],[255, 255, 255])]
    for (lower, upper) in white_boundary:
        lower_arr = np.array(lower)
        upper_arr = np.array(upper)
        present = cv2.inRange(img, lower_arr, upper_arr)
        ratio_color = cv2.countNonZero(present)/(img.size/3)
        return (100-np.round(ratio_color*100, 2))/100

# Calculate land use percentage within each raster JPG
def find_land_use_percentage(image):
    img_data_list = []
    new_perc = remove_margin(image) # remove margin of image clip
    for color in color_array:
        boundaries = [([color[3]-diff, color[2]-diff, color[1]-diff],[color[3]+diff, color[2]+diff, color[1]+diff])]
        for (lower, upper) in boundaries:
            lower_arr = np.array(lower)
            upper_arr = np.array(upper)
            present = cv2.inRange(image, lower_arr, upper_arr)
            ratio_color = cv2.countNonZero(present)/(image.size/3)
            img_data_list.append(np.round((ratio_color*100)/new_perc, 2))
    data_list.append(img_data_list) # Add each image's raster percentage to data list

# Process each Image

In [86]:
# Read in images with CV2 and run through land use function
def process(image):
    img = cv2.imread(image) # test image
    find_land_use_percentage(img)
    
for image in img_list:
    process(image)

# Create DataFrame from Land Use Percentage

In [87]:
df = pd.DataFrame(data_list, columns=['Impervious Road (%)', 'Impervious Non-road (%)', 'Fractional Turf (%)', 'Cropland/Pasture (%)', 'Fractional Impervious (%)', 'Tidal Wetland (%)', 'Floodplain Wetland (%)', 'Forest (%)', 'Mixed Open (%)', 'Tree Cover Over Turf (%)', 'Tree Cover Over Impervious (%)', 'Water (%)'])
df = df.drop(['Tidal Wetland (%)','Fractional Impervious (%)', 'Floodplain Wetland (%)','Tree Cover Over Turf (%)', 'Tree Cover Over Impervious (%)'], axis=1)
df = df.rename(index={0: "Lower Sassafras River", 1: "Langford Creek", 2: "Severn River-Frontal Chesapeake Bay", 3: "Parkers Creek-Frontal Chesapeake Bay", 4: "Saint Leonard Creek-Patuxent River", 5: "Fox Creek-Honga River", 6: "Saint Jerome Creek-Frontal Chesapeake Bay" , 7: "Cockrell Creek-Frontal Chesapeake Bay"})
df.index.name = 'Name of HUC'
df

,Impervious Road (%),Impervious Non-road (%),Fractional Turf (%),Cropland/Pasture (%),Forest (%),Mixed Open (%),Water (%)
Name of HUC,,,,,,,
Lower Sassafras River,0.63,0.06,0.57,45.75,16.11,0.25,5.42
Langford Creek,0.23,0.01,0.11,58.73,14.63,0.09,3.77
Severn River-Frontal Chesapeake Bay,2.67,5.65,7.92,0.06,11.58,0.78,34.70
Parkers Creek-Frontal Chesapeake Bay,0.36,0.36,0.00,0.74,17.22,0.07,70.25
Saint Leonard Creek-Patuxent River,0.81,1.10,0.73,20.04,42.84,1.08,3.11
Fox Creek-Honga River,0.02,0.03,0.00,0.05,0.61,0.05,83.26
Saint Jerome Creek-Frontal Chesapeake Bay,0.33,0.06,0.08,8.32,49.12,0.02,4.55
Cockrell Creek-Frontal Chesapeake Bay,1.58,0.50,0.00,5.51,32.89,0.43,18.94
